In [1]:
import math,csv, random, collections
import numpy as np
import pickle
import seaborn as sns
import matplotlib.patches as mpatches
import random, string

import imp, sys
sys.path.insert(0, '../')
import utils, algorithms
import h5py, imp

import matplotlib.pyplot as plt
from IPython.display import display, HTML
from multiprocessing import Pool
from multiprocessing import Process, Queue
imp.reload(utils)
imp.reload(algorithms)
        
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
sns.set_style('dark')
color_p = sns.color_palette("hls", 10*2)

In [2]:
def BP(M, n_steps, filename):
    n = M.shape[0]
    BP_matrix = np.zeros((2, n, n)) #BP_matrix[i] will store the state of BP in step i
    BP_matrix[0] = M.copy() #First state of BP_matrix is the data matrix itself
    accuracy = np.zeros(n_steps) #Stores accuracy after each step
    
    #True answer
    true_answer = np.zeros((n,2))
    true_answer[:,0] = np.roll(np.arange(n),1)
    true_answer[:,1] = np.roll(np.arange(n),-1)
    true_answer = true_answer.astype(int)

    for l in range(1, n_steps):
        for i in range(n):
            for j in range(n):
                row = BP_matrix[(l-1)%2][:,i].copy()
                row[j] = np.inf
                BP_matrix[l%2][i][j] = M[i][j] - np.partition(row, 1)[1]
                
        BP_matrix[l%2][np.diag_indices_from(BP_matrix[l%2])] = np.inf
        
        M_final = BP_matrix[l%2]
        M_final[np.diag_indices_from(M_final)] = np.inf
        answer = M_final.argsort(axis=1)[:,:2]
        answer.sort(axis=1)
        answer[0] = answer[0][::-1]
        answer[-1] = answer[-1][::-1]
        accuracy[l] = np.mean(answer==true_answer)
        if accuracy[l] == 1:
            accuracy[l:] = accuracy[l]
            break
        if l > 50 and abs(np.mean(accuracy[l-25:l]) - np.mean(accuracy[l-50:l-25])) < 0.001: 
            accuracy[l:] = np.mean(accuracy[l-25:l])
            break
    output = ','.join(np.array(accuracy[1:]).astype(str))
    output = str(n)+","+str(mu)+","+str(l)+","+output
    fd = open(filename,'a')
    fd.write(output+"\n")
    fd.close()


def LP(data, file):
    n = data.shape[0]
    model , t      = algorithms.Solve_LP(data)
    answer         = np.array(model.X).reshape(n,n)
    correct_answer = algorithms.correct_data_answer(data,k=1)
           
    error = (np.where(correct_answer-answer!=0)[0].shape[0]+0.0)/(2*n)
    fp = (np.where(correct_answer-answer==-1)[0].shape[0]+0.0)/(2*n)
    point5 = (np.where(answer==0.5)[0].shape[0]+0.0)/(2*n)
    if file != None:
        fd = open(file,'a')
        fd.write(', '.join([str(n), str(mu), str(error), str(fp), str(point5),"\n"]))
        fd.close()
    

In [4]:
n = 1000
it_limit = np.sqrt(4*np.log(n))
mg_limit = np.sqrt(6*np.log(n))
sg_limit = np.sqrt(8*np.log(n))
mu_array1 = np.linspace(it_limit*.8,it_limit*1.2,8)
mu_array2 = np.linspace(it_limit*.95,it_limit*1.05,8)
mu_array3 = np.linspace(it_limit*1.05,it_limit*1.2,8)
mu_array = np.concatenate([mu_array1, mu_array2, mu_array3])
mu_array.sort()
# M = M.astype(int)
print "it = ", it_limit
print "mg = ", mg_limit
print "sg = ", sg_limit
print mu_array
print mu_array.shape[0]
no_exp = 50

it =  5.25652176976
mg =  6.43789807887
sg =  7.4338443777
[ 4.20521742  4.50559009  4.80596276  4.99369568  5.06878885  5.10633543
  5.14388202  5.21897519  5.29406835  5.36916152  5.40670811  5.44425469
  5.51934786  5.51934786  5.63198761  5.70708078  5.74462736  5.85726711
  5.96990687  6.00745345  6.08254662  6.19518637  6.30782612  6.30782612]
24


In [ ]:
filenameBP = 'data/2BP_'+str(n)+'.csv'   
filenameLP = 'data/2LP_'+str(n)+'.csv'   

for mu in mu_array:
    print "\nmu = ", mu, "Experiment = "
    no_process = 32
    pool = Pool(processes=no_process)  
    processes = []
    for exp in range(no_exp/2):
        print " ", exp+1,
#         A = np.zeros([n,n])
#         A[utils.kth_diag_indices(A,1)] = mu
#         A[utils.kth_diag_indices(A,-1)] = mu
        M = algorithms.Gaussian_matrix(mu=mu, n =n)
        #BP
        M[np.diag_indices_from(M)]= -np.inf
        args = (-M, n*10, filenameBP)
        p = Process(target = BP, args = args)
        p.start()
        processes.append(p)
        
        #LP
        M[np.diag_indices_from(M)]= 0
        args = (M, filenameLP)
        p = Process(target = LP, args = args)
        p.start()
        processes.append(p)

    print  "\nJoining Process"
    for j, p in enumerate(processes):
        print j, "\t",
        p.join()



mu =  4.20521741581 Experiment = 
  1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25 
Joining Process
0 	

In [ ]:
filenameBP = 'data/BP_'+str(n)+'.csv'   
filenameLP = 'data/LP_'+str(n)+'.csv'   

for mu in mu_array:
    print "\nmu = ", mu, "Experiment = "
    no_process = 32
    pool = Pool(processes=no_process)  
    processes = []
    for exp in range(no_exp/2, no_exp):
        print " ", exp+1,
        M = M_list[exp]+np.eye(n)*mu
        #BP
        M[np.diag_indices_from(M)]= -np.inf
        args = (-M, n*10, filenameBP)
        p = Process(target = BP, args = args)
        p.start()
        processes.append(p)
        
        #LP
        M[np.diag_indices_from(M)]= 0
        args = (M, filenameLP)
        p = Process(target = LP, args = args)
        p.start()
        processes.append(p)

    print  "\nJoining Process"
    for j, p in enumerate(processes):
        print j, "\t",
        p.join()
